In [ ]:
import intake
import xarray as xr
import regionmask
import matplotlib.pyplot as plt
from xmip.preprocessing import combined_preprocessing
import numpy as np
import dask
from xmip.regionmask import merged_mask
import glob
import os

basins = regionmask.defined_regions.natural_earth_v4_1_0.ocean_basins_50

In [ ]:
# Get the THETAO data files from saved
thetaofiles = glob.glob('./thetao-models-1990-2000/*.nc')
thetaofiles.sort()

# Get the AREACELLO data files from saved
areacellofiles = glob.glob('./areacello-multimodel/*.nc')
areacellofiles.sort()

In [ ]:
## Setting figure configurations
fig, ax = plt.subplots(1, 1,constrained_layout=True, figsize=(10,15))

legendValues = []

##loop through saved netCDF THETAO and AREACELLO datasets
for thetaofile, areacellofile in zip(thetaofiles, areacellofiles):
    
    #obtain source_id from file
    modelName = thetaofile.split('_')[1]
   
    #open datasets using XArrays     
    thetao_dataset = xr.open_dataset(thetaofile)
    areacello_dataset = xr.open_dataset(areacellofile)
    
    #Select the basins     
    mask = merged_mask(basins, thetao_dataset)        
    
    #Append model names to figure         
    legendValues.append(modelName)
    
    #Select mask ocean regions     
    ds_masked = thetao_dataset.where(np.logical_or(np.logical_or(mask == 2, mask==3),mask==4))
    area_masked = areacello_dataset.where(np.logical_or(np.logical_or(mask == 2, mask==3),mask==4))
    
    #Obtain a weighted mean for the masked Thetao and compute     
    yy = (ds_masked.thetao[0,:,:,:] * area_masked.areacello[0,:,:]).sum(['x', 'y'])/area_masked.areacello[0,:,:].sum(['x', 'y'])
    yy.compute()
    
    # Plot the vertical temperature profile     
    yy.plot(y='lev', ax=ax, add_legend=True)
    
    plt.legend(legendValues, loc='lower right')
    
    plt.title('Historical Model Comparison 1990 - 2000')
    
plt.gca().invert_yaxis()

plt.savefig('basin.png', dpi=300)